第七章 函数装饰器和闭包

7.1 装饰器基础知识

In [3]:
'''
假如有个名为decorate的装饰器
@decorate
def target():
    print('running target())
    
相当于
def target():
    print('running target()')
target=decorate(target)
'''

"\n假如有个名为decorate的装饰器\n@decorate\ndef target():\n    print('running target())\n    \n相当于\ndef target():\n    print('running target()')\ntarget=decorate(target)\n"

In [4]:
def deco(func):
    def inner():
        print('running inner()')
    return inner

@deco
def target():
    print('running target()')

In [5]:
target()

running inner()


In [6]:
target  #target现在其实是inner的引用

<function __main__.deco.<locals>.inner>

7.2 python何时执行装饰器

装饰器的一个关键特性是，它们在被装饰的函数定义之后立即执行

In [7]:
registry = []  # <1>

def register(func):  # <2>
    print('running register(%s)' % func)  # <3>
    registry.append(func)  # <4>
    return func  # <5>

@register  # <6>
def f1():
    print('running f1()')

@register
def f2():
    print('running f2()')

def f3():  # <7>
    print('running f3()')

def main():  # <8>
    print('running main()')
    print('registry ->', registry)
    f1()
    f2()
    f3()

if __name__=='__main__':
    main()  # <9>

# END REGISTRATION

running register(<function f1 at 0x0000021734FF8158>)
running register(<function f2 at 0x0000021734FF8378>)
running main()
registry -> [<function f1 at 0x0000021734FF8158>, <function f2 at 0x0000021734FF8378>]
running f1()
running f2()
running f3()


In [17]:
import sys
import os
sys.path.insert(0,r'D:\WORKSPACE2\python35\python8.25\fluent_python')


如果导入registration.py模块，输出如下

In [16]:
import registration

running register(<function f1 at 0x0000021735066158>)
running register(<function f2 at 0x0000021735066620>)


注意，上例说明，函数装饰器在导入模块时立即执行，而被装饰的函数只有在明确调用时运行。这突出了python程序员所说的导入时和运行时之间的区别

7.3 使用装饰器改进’策略‘模式

回顾上一章的电商促销折扣示例。定义体中有函数名称，但是best_promo用来判断哪个折扣幅度最大的promos列表也有函数名称。这种重复是个问题，因为新增的策略函数可能忘记添加到列表中。使用装饰器解决该问题。

In [18]:
from collections import namedtuple

Customer = namedtuple('Customer', 'name fidelity')


class LineItem:

    def __init__(self, product, quantity, price):
        self.product = product
        self.quantity = quantity
        self.price = price

    def total(self):
        return self.price * self.quantity


class Order:  # the Context

    def __init__(self, customer, cart, promotion=None):
        self.customer = customer
        self.cart = list(cart)
        self.promotion = promotion

    def total(self):
        if not hasattr(self, '__total'):
            self.__total = sum(item.total() for item in self.cart)
        return self.__total

    def due(self):
        if self.promotion is None:
            discount = 0
        else:
            discount = self.promotion(self)
        return self.total() - discount

    def __repr__(self):
        fmt = '<Order total: {:.2f} due: {:.2f}>'
        return fmt.format(self.total(), self.due())

promos = []

def promotion(promo_func):
    promos.append(promo_func)
    return promo_func

@promotion
def fidelity(order):
    return order.total()*0.05 if order.customer.fidelity >= 1000 else 0

@promotion
def bulk_item(order):
    discount=0
    for item in order.cart:
        if item.quantity >= 20:
            discount+=item.total()*0.1
    return discount

@promotion
def large_order(order):
    distinct_items = {item.product for item in order.cart}
    if len(distinct_items) >=10:
        return order.total()*0.07
    return 0

def best_promo(order):
    return max(promo(order) for promo in promos)

7.4 变量作用域规则

In [1]:
#实例7-4
def f1(a):
    print(a)
    print(b)

In [2]:
f1(3)

3


NameError: name 'b' is not defined

In [3]:
b = 6
f1(3)

3
6


In [4]:
#示例7-5 b是局部变量，因为在函数的定义体中给他赋值了
b = 6
def f2(a):
    print(a)
    print(b)
    b=9

In [5]:
f2(3)

3


UnboundLocalError: local variable 'b' referenced before assignment

python不要求声明变量，但是假定在函数定义体中赋值的变量是局部变量

如果在函数值赋值时想让解释器把b当成全局变量，要使用global声明

In [6]:
b=6
def f3(a):
    global b
    print(a)
    print(b)
    b=9

In [7]:
f3(3)

3
6


In [8]:
b

9

In [9]:
f3(3)

3
9


In [10]:
b=30
f3(3)

3
30


In [11]:
#dis模块为反汇编python函数字节码提供了简单的方式。
from dis import dis
dis(f1)

  2           0 LOAD_GLOBAL              0 (print)
              3 LOAD_FAST                0 (a)
              6 CALL_FUNCTION            1 (1 positional, 0 keyword pair)
              9 POP_TOP

  3          10 LOAD_GLOBAL              0 (print)
             13 LOAD_GLOBAL              1 (b)
             16 CALL_FUNCTION            1 (1 positional, 0 keyword pair)
             19 POP_TOP
             20 LOAD_CONST               0 (None)
             23 RETURN_VALUE


In [12]:
dis(f2)

  4           0 LOAD_GLOBAL              0 (print)
              3 LOAD_FAST                0 (a)
              6 CALL_FUNCTION            1 (1 positional, 0 keyword pair)
              9 POP_TOP

  5          10 LOAD_GLOBAL              0 (print)
             13 LOAD_FAST                1 (b)
             16 CALL_FUNCTION            1 (1 positional, 0 keyword pair)
             19 POP_TOP

  6          20 LOAD_CONST               1 (9)
             23 STORE_FAST               1 (b)
             26 LOAD_CONST               0 (None)
             29 RETURN_VALUE


7.5 闭包

闭包指延伸了作用域的函数，其中包含函数定义体中引用、但是不在定义体中定义的非全局变量关键是函数能访问定义体之外定义的非全局变量

In [14]:
#计算移动平均值的类
class Averager():
    
    def __init__(self):
        self.series=[]
    
    def __call__(self, new_value):
        self.series.append(new_value)
        total = sum(self.series)
        return total/len(self.series)

In [15]:
avg =Averager()

In [16]:
avg(10)

10.0

In [17]:
avg(11)

10.5

In [18]:
avg(12)

11.0

In [19]:
#计算移动平均值的高阶函数
def make_averager():
    series = []
    
    def averager(new_value):
        series.append(new_value)
        total = sum(series)
        return total/len(series)
    return averager

调用make_averager时，返回一个averager函数对象

In [20]:
avg = make_averager()
avg(10)

In [21]:
avg(11)

10.5

series是make_averager函数的局部变量，因为那个函数的定义体中已经初始化了series：series=[]。可是，调用avg(10)时，make_averager函数已经返回了，而它的本地作用域已经一去不复返了。

在averager函数中，series是自由变量。指未在本地作用域中绑定的变量。

In [22]:
avg.__code__.co_varnames

('new_value', 'total')

In [23]:
avg.__code__.co_freevars

('series',)

In [24]:
avg.__closure__

(<cell at 0x000002DC4E2986D8: list object at 0x000002DC4E229FC8>,)

In [26]:
#series的绑定在返回的avg函数的__closure__属性中。avg.__closure__中的各个元素对应于avg.__code__.co_freevars中的一个名称。
#这些元素是cell对象，有个cell_contents属性，保存着真正的值
avg.__closure__[0].cell_contents

[10, 11]

In [28]:
avg.__closure__[1]

IndexError: tuple index out of range

综上，闭包是一种函数，它会保留定义函数时存在的自由变量的绑定，这样使用函数时，虽然定义作用域不可用了，但是仍能使用那些绑定。

注意，只有嵌套在其它函数中的函数才可能需要处理不在全局作用域中的外部变量。

7.6 nonlocal声明

In [29]:
#计算移动平均值的高阶函数，不保存所有历史值，但有缺陷
def make_averager():
    count=0
    total=0
    
    def averager(new_value):
        count+=1
        total+=new_value
        return total/count
    return averager

In [30]:
avg=make_averager()
avg(10)

UnboundLocalError: local variable 'count' referenced before assignment

count+=1,隐式的创建了局部变量，将count变为局部变量

In [31]:
#计算移动平均值的高阶函数，不保存所有历史值，使用nonlocal修正
def make_averager():
    count=0
    total=0
    
    def averager(new_value):
        nonlocal count,total
        count+=1
        total+=new_value
        return total/count
    return averager

In [32]:
avg=make_averager()
avg(10)

10.0

7.7 实现一个简单的装饰器

In [53]:
import time
def clock(func):
    print('1')
    def clocked(*args,**kwargs):
        print(['%s=%r' %(k,w) for k,w in kwargs.items()])
        t0 = time.time()
        result = func(*args)
        elapsed = time.time() - t0
        name = func.__name__
        arg_str = ', '.join(repr(arg) for arg in args)
        print('[%0.8fs] %s(%s) -> %r' % (elapsed, name, arg_str, result))
        return result
    return clocked
    

In [55]:
import time
@clock
def snooze(seconds,ire='aa'): #ire='aa'并未传到clocked
    time.sleep(seconds)

@clock
def factorial(n):
    return 1 if n < 2 else n*factorial(n-1)

if __name__=='__main__':
    print('*' * 40, 'Calling snooze(.123)')
    snooze(.123)
    print('*' * 40, 'Calling factorial(6)')
    print('6! =', factorial(6))

1
1
**************************************** Calling snooze(.123)
[]
[0.12370706s] snooze(0.123) -> None
**************************************** Calling factorial(6)
[]
[]
[]
[]
[]
[]
[0.00000000s] factorial(1) -> 1
[0.00000000s] factorial(2) -> 2
[0.00000000s] factorial(3) -> 6
[0.00000000s] factorial(4) -> 24
[0.00000000s] factorial(5) -> 120
[0.00000000s] factorial(6) -> 720
6! = 720


In [56]:
factorial.__name__ #factorial保存的是clocked函数的引用

'clocked'

此装饰器有几个缺点：不支持关键字参数，而且遮盖了被装饰函数的__name__和__doc__属性。下例使用functools.wraps装饰器把相关的属性从func复制到clocked中

In [2]:
import time
import functools
def clock(func):
    @functools.wraps(func)
    def clocked(*args,**kwargs):
        t0 = time.time()
        result = func(*args,**kwargs)
        elapsed = time.time() - t0
        name = func.__name__
        arg_list=[]
        if args:
            arg_list.append(', '.join(repr(arg) for arg in args))
        if kwargs:
            pairs = ['%s=%r'%(k,w) for k,w in sorted(kwargs.items())]
            arg_list.append(', '.join(pairs))
        arg_str = ', '.join(arg_list)
        print('[%0.8fs] %s(%s) -> %r' % (elapsed, name, arg_str, result))
        return result
    return clocked


In [3]:
import time
@clock
def snooze(seconds,ire='aa'): #ire='aa'并未传到clocked
    time.sleep(seconds)

@clock
def factorial(n):
    return 1 if n < 2 else n*factorial(n-1)

if __name__=='__main__':
    print('*' * 40, 'Calling snooze(.123)')
    snooze(.123)
    print('*' * 40, 'Calling factorial(6)')
    print('6! =', factorial(6))

**************************************** Calling snooze(.123)
[0.12335372s] snooze(0.123) -> None
**************************************** Calling factorial(6)
[0.00000000s] factorial(1) -> 1
[0.00000000s] factorial(2) -> 2
[0.00000000s] factorial(3) -> 6
[0.00000000s] factorial(4) -> 24
[0.00000000s] factorial(5) -> 120
[0.00000000s] factorial(6) -> 720
6! = 720


In [4]:
factorial.__name__

'factorial'

7.8 标准库中的装饰器

7.8.1 使用functools.lru_cache做备忘

In [5]:
@clock
def fibonacci(n):
    if n<2:
        return n
    else:
        return fibonacci(n-2)+fibonacci(n-1)
if __name__ == '__main__':
    print(fibonacci(6))

[0.00000000s] fibonacci(0) -> 0
[0.00000000s] fibonacci(1) -> 1
[0.00097656s] fibonacci(2) -> 1
[0.00000000s] fibonacci(1) -> 1
[0.00000000s] fibonacci(0) -> 0
[0.00000000s] fibonacci(1) -> 1
[0.00000000s] fibonacci(2) -> 1
[0.00000000s] fibonacci(3) -> 2
[0.00097656s] fibonacci(4) -> 3
[0.00000000s] fibonacci(1) -> 1
[0.00000000s] fibonacci(0) -> 0
[0.00000000s] fibonacci(1) -> 1
[0.00000000s] fibonacci(2) -> 1
[0.00000000s] fibonacci(3) -> 2
[0.00000000s] fibonacci(0) -> 0
[0.00000000s] fibonacci(1) -> 1
[0.00000000s] fibonacci(2) -> 1
[0.00000000s] fibonacci(1) -> 1
[0.00000000s] fibonacci(0) -> 0
[0.00000000s] fibonacci(1) -> 1
[0.00000000s] fibonacci(2) -> 1
[0.00000000s] fibonacci(3) -> 2
[0.00000000s] fibonacci(4) -> 3
[0.00000000s] fibonacci(5) -> 5
[0.00097656s] fibonacci(6) -> 8
8


In [6]:
@functools.lru_cache()
@clock
def fibonacci(n):
    if n < 2:
        return n
    return fibonacci(n-1)+fibonacci(n-2)

if __name__ == '__main__':
    print(fibonacci(6))

[0.00000000s] fibonacci(1) -> 1
[0.00000000s] fibonacci(0) -> 0
[0.00000000s] fibonacci(2) -> 1
[0.00000000s] fibonacci(3) -> 2
[0.00000000s] fibonacci(4) -> 3
[0.00000000s] fibonacci(5) -> 5
[0.00000000s] fibonacci(6) -> 8
8


In [8]:
functools.lru_cache(maxsize=128,typed=False)

<function functools.lru_cache.<locals>.decorating_function>

7.8.2 单分派泛函数

In [1]:
import html

def htmlize(obj):
    content = html.escape(repr(obj))
    return '<pre>{}</pre>'.format(content)


In [2]:
htmlize({1,2,3})

'<pre>{1, 2, 3}</pre>'

In [3]:
htmlize(abs)

'<pre>&lt;built-in function abs&gt;</pre>'

In [4]:
htmlize(42)

'<pre>42</pre>'

In [5]:
#singledispatch创建一个自定义的htmlize.register装饰器，把多个函数绑定在一起组成一个泛函数

from functools import singledispatch
from collections import  abc
import numbers
import html

@singledispatch #@singledispatch标记处理object类型的基函数
def htmlize(obj):
    content = html.escape(repr(obj))
    return '<pre>{}</pre>'.format(content)

@htmlize.register(str) #各个专门函数使用@base_function.register(type)装饰
def _(text):#专门函数的名称无关紧要，_是个不错的选择，简单明了
    content=html.escape(text).replace('\n','<br>\n')
    return '<p>{0}</p>'.format(content)

@htmlize.register(numbers.Integral) #numbers.Integral是int的虚拟超类
def _(n):
    return '<pre>{0}(0x{0:x})</pre>'.format(n)

@htmlize.register(tuple)#可以叠放几个register装饰器，让同一个函数支持不同类型
@htmlize.register(abc.MutableSequence)
def _(seq):
    inner = '</li>\n<li>'.join(htmlize(item) for item in seq)
    return '<ul>\n<li>'+inner+'</li>\n</ul>'

In [6]:
htmlize([1,2,3])

'<ul>\n<li><pre>1(0x1)</pre></li>\n<li><pre>2(0x2)</pre></li>\n<li><pre>3(0x3)</pre></li>\n</ul>'

In [8]:
htmlize((1,2,3))

'<ul>\n<li><pre>1(0x1)</pre></li>\n<li><pre>2(0x2)</pre></li>\n<li><pre>3(0x3)</pre></li>\n</ul>'

In [7]:
htmlize('wjy')

'<p>wjy</p>'

7.9 叠放装饰器

In [8]:
'''
@d1
@d2
def f():
    print('f')
等同于：
def f():
    print('f')
f1=d1(d2(f))

'''

7.10参数化装饰器

In [9]:
registry=[]
def register(func):
    print('running register(%s)'%func)
    registry.append(func)
    return func

@register
def f1():
    print('running f1()')

print('running main()')
print('registry ->',registry)
f1()

running register(<function f1 at 0x0000020A5985BC80>)
running main()
registry -> [<function f1 at 0x0000020A5985BC80>]
running f1()


7.10.1 一个参数化的注册装饰器

In [23]:
registry=set() #registry现在是一个set对象，这样添加或删除函数的速度更快
def register(active=True):#register接受一个可选的关键字参数
    def decorate(func):#decorate这个内部函数是真正的装饰器，注意，它的参数是一个函数
        print('running register(active=%s->decorate(%s)'%(active,func))
        if active:#从闭包中获取
            registry.add(func)
        else:
            registry.discard(func)
            
        return func#decora是一个装饰器，必须返回一个函数
    return decorate#register是装饰器工厂函数，因此返回decorate

#这里的关键是，register()要返回decorate，然后把它应用到被装饰的函数上

@register(active=False)#@register工厂函数必须作为函数调用
def f1():
    print('running f1()')

@register()
def f2():
    print('running f2()')

def f3():
    print('running f3()')

running register(active=False->decorate(<function f1 at 0x0000020A59908510>)
running register(active=True->decorate(<function f2 at 0x0000020A598FFE18>)


In [14]:
register()

<function __main__.register.<locals>.decorate>

In [15]:
registry

{<function __main__.f2>}

In [16]:
f2.__name__

'f2'

In [24]:
register()(f3)

running register(active=True->decorate(<function f3 at 0x0000020A598FFEA0>)


<function __main__.f3>

In [25]:
registry

{<function __main__.f2>, <function __main__.f3>}

In [26]:
register(active=False)(f2)

running register(active=False->decorate(<function f2 at 0x0000020A598FFE18>)


In [27]:
registry

{<function __main__.f3>}

7.10.2 参数化clock装饰圈

In [28]:
#添加一个功能：让用户传入一个格式字符串，控制被装饰函数的输出
import time
default_fmt = '[{elapsed:0.8f}s] {name}({args}) -> {result}'

def clock(fmt=default_fmt):
    def decorate(func):
        def clocked(*_args):
            t0 = time.time()
            _result=func(*_args)
            elapsed=time.time()-t0
            name=func.__name__
            args=', '.join(repr(arg) for arg in _args)
            result = repr(_result)#result是_result的字符串表示
            print(fmt.format(**locals()))#这里使用**locals()是为了在fmt中引用clocked的局部变量
            return _result
        return clocked
    return decorate

if __name__ == '__main__':
    @clock()
    def snooze(seconds):
        time.sleep(seconds)
    
    for i in range(3):
        snooze(.123)

[0.12392402s] snooze(0.123) -> None
[0.12321234s] snooze(0.123) -> None


[0.12358737s] snooze(0.123) -> None
